In [700]:
'''
bash script for cpu top stats 
'''
#script 
'''
#!/bin/bash 
filename='cpu_stat.txt'
n=2
while read line; do
echo "line changed: $n " ;
tail -n $n cpu_stat.txt  | sed -n '8, 12{s/^ *//;s/ *$//;s/  */,/gp;};12q' >> out.txt
n=$((n+1))
done < $filename


'''


'\n#!/bin/bash \nfilename=\'cpu_stat.txt\'\nn=2\nwhile read line; do\necho "line changed: $n " ;\ntail -n $n cpu_stat.txt  | sed -n \'8, 12{s/^ *//;s/ *$//;s/  */,/gp;};12q\' >> out.txt\nn=$((n+1))\ndone < $filename\n\n\n'

In [701]:
import pandas as pd
import numpy as np
import re
import os
import glob

In [702]:
cpulog = "/home/jahanzeb/rdma_project/converted1gb.csv"
cpufd = open(cpulog,"r")
#' Manually add first row' 
sender_log = "/home/jahanzeb/rdma_project/1gb_client_stats.csv"
senderfd = open(sender_log,"rt")

In [703]:
'''
'file_number','host_ip','host_port','slab_buf_bytes',
'slab_order','slab_partition',
                                         'bandwidth','file_size_bytes',
                                         'time_taken_sec','overall_cpu'

'''

sender_df = pd.read_csv(senderfd)
cpu_df = pd.read_csv(cpufd)

/home/jahanzeb/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (0,2,3,6,8,9) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [704]:
len(cpu_df)

96080

In [705]:
len(sender_df)

100

In [706]:
cpu_df.head(96080)

,PID,USER,PR,NI,VIRT,RES,SHR,S,%CPU,%MEM,TIME+,COMMAND
0,30634,root,20,0,12.078g,3.627g,3460,R,86.7,1.2,31:41.71,rxfer_test
1,30634,root,20,0,12.078g,4.034g,3460,S,13.3,1.3,31:41.58,rxfer_test
2,30634,root,20,0,12.078g,3.627g,3460,R,86.7,1.2,31:41.71,rxfer_test
3,30634,root,20,0,12.078g,4.034g,3460,S,6.7,1.3,31:41.55,rxfer_test
4,30634,root,20,0,12.078g,4.034g,3460,S,13.3,1.3,31:41.58,rxfer_test
...,...,...,...,...,...,...,...,...,...,...,...,...
96075,24660,root,20,0,4386728,4.004g,3420,S,6.70,1.3,0:01.59,rxfer_test
96076,24660,root,20,0,4386728,4.004g,3420,S,66.70,1.3,0:01.69,rxfer_test
96077,24660,root,20,0,1240976,383356,3420,R,100.00,0.1,0:01.85,rxfer_test
96078,24660,root,20,0,4386868,366272,3484,R,93.30,0.1,0:02.00,rxfer_test


In [707]:
divide_sc = len(cpu_df)/len(sender_df)
divide_sc = int(divide_sc)
divide_sc

960

In [708]:
cpumdf = cpu_df[['%CPU','%MEM']].iloc[0:50].mean()

In [731]:
cpu_df.head(96080)

cpu_df.at[65295,'%CPU']

cpu_df['%CPU']= pd.to_numeric(cpu_df['%CPU'], errors='coerce')
cpu_df['%MEM']= pd.to_numeric(cpu_df['%MEM'], errors='coerce')

#cpu_df[['%CPU']].iloc[65282:67000].mean()

In [732]:
#cpu_df.iloc[0:3]
#cpu_df.head(65282)

#cpu_df[['%CPU']].iloc[65282:67000].mean(axis=0, skipna=False)

In [734]:
cpumdf = pd.DataFrame()
temp = 0
i = 0
temp1 = divide_sc
count = 0
while True:
    if(i==len(cpu_df)):
        break
    else:
        if i == 0:
            cpu_ptg = cpu_df[['%CPU']].iloc[temp:temp1].mean()
            mem_ptg = cpu_df[['%MEM']].iloc[temp:temp1].mean()
                
            cpumdf[count]= cpu_df[['%CPU','%MEM']].iloc[0:temp1].mean()
            #cpumdf.apply(cpu_df.to_numeric, args=['%CPU','%MEM']).mean(axis=0, skipna=False)
            print(f"Row Being Processed From {temp} to {temp1} CPU : {cpu_ptg}% | MEM: {mem_ptg}%")    
            #cpumdf[count]= cpu_df[['%CPU','%MEM']].iloc[0:temp1].mean()
            #cpumdf= cpu_df[['%CPU','%MEM']]
            #cpumdf.append(cpu_df[['%CPU','%MEM']].iloc[0:temp1].mean(),ignore_index =True)
            #pd.concat((cpumdf,cpu_df[['%CPU','%MEM']].iloc[0:temp1].mean()))
            #print(f"Row Being Processed From {temp} to {temp1}")
            temp = temp1
            count = count + 1
            print("count : ",count)
        else:
            if i==temp1:
                temp = temp1
                temp = temp + 1
                temp1 = temp1 + divide_sc
                cpu_ptg = cpu_df[['%CPU']].iloc[temp:temp1].mean()
                mem_ptg = cpu_df[['%MEM']].iloc[temp:temp1].mean()
                #cpumdf[count]['%CPU'] = cpu_ptg
                cpumdf[count] = cpu_df[['%CPU','%MEM']].iloc[temp:temp1].mean()
                #cpumdf.apply(cpu_df.to_numeric, args=['%CPU','%MEM']).mean(axis=0, skipna=False)
                
                #cpumdf= cpu_df[['%CPU','%MEM']]
                #pd.concat((cpumdf,cpu_df[['%CPU','%MEM']].iloc[temp:temp1].mean()))
                print(f"Row Being Processed From {temp} to {temp1} CPU : {cpu_ptg}% | MEM: {mem_ptg}%")
                count = count + 1
                print("count : ",count)
        
        i = i + 1
        
        
        
    
    

Row Being Processed From 0 to 960 CPU : %CPU    63.752396
dtype: float64% | MEM: %MEM    0.943125
dtype: float64%
count :  1
Row Being Processed From 961 to 1920 CPU : %CPU    65.253389
dtype: float64% | MEM: %MEM    0.925443
dtype: float64%
count :  2
Row Being Processed From 1921 to 2880 CPU : %CPU    62.988425
dtype: float64% | MEM: %MEM    0.941397
dtype: float64%
count :  3
Row Being Processed From 2881 to 3840 CPU : %CPU    64.137331
dtype: float64% | MEM: %MEM    0.925443
dtype: float64%
count :  4
Row Being Processed From 3841 to 4800 CPU : %CPU    67.037435
dtype: float64% | MEM: %MEM    0.91585
dtype: float64%
count :  5
Row Being Processed From 4801 to 5760 CPU : %CPU    63.416997
dtype: float64% | MEM: %MEM    0.945881
dtype: float64%
count :  6
Row Being Processed From 5761 to 6720 CPU : %CPU    66.837748
dtype: float64% | MEM: %MEM    0.914807
dtype: float64%
count :  7
Row Being Processed From 6721 to 7680 CPU : %CPU    66.130344
dtype: float64% | MEM: %MEM    0.920438
d

/tmp/ipykernel_2398239/131267877.py:33: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  cpumdf[count] = cpu_df[['%CPU','%MEM']].iloc[temp:temp1].mean()


In [735]:
cpumdf

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,100
%CPU,63.752396,65.253389,62.988425,64.137331,67.037435,63.416997,66.837748,66.130344,63.095620,60.832742,...,61.383733,64.244734,76.911950,90.005527,89.670386,89.96830,89.788217,89.984984,90.572367,87.706329
%MEM,0.943125,0.925443,0.941397,0.925443,0.915850,0.945881,0.914807,0.920438,0.948801,0.946715,...,0.944943,0.930136,0.843711,0.751512,0.744526,0.75829,0.755474,0.748488,0.745360,0.702532


In [736]:
sender_df['overall_mem'] = None

In [737]:
sender_df['overall_cpu'] = None

#cpumdf.iloc[1,[0]]

In [738]:
#sender_df.loc[[0]]['overall_cpu'] = cpumdf.iloc[1,[0]]

In [739]:
#sender_df.iloc[[0]]['overall_cpu'] = 0

#sender_df.at[0,'overall_cpu'] = cpumdf.iloc[1,[0]]

In [740]:
sender_df.head(3)

,file_number,host_ip,host_port,slab_buf_bytes,slab_partition,bandwidth,file_size_bytes,time_taken_sec,overall_cpu,overall_mem
0,0,10.0.0.50,5202,1073741824,4,34.56 Gb,1.073742e+09,0.248558,None,None
1,1,10.0.0.50,5202,1073741824,4,34.54 Gb,1.073742e+09,0.248713,None,None
2,2,10.0.0.50,5202,1073741824,4,34.95 Gb,1.073742e+09,0.245784,None,None


In [741]:


for i in range(0,len(sender_df)):
    if i>=9997:
        break
    else:
        sender_df.at[i,'overall_cpu'] = cpumdf[i]['%CPU']
        sender_df.at[i,'overall_mem'] = cpumdf[i]['%MEM']

In [742]:
sender_df

,file_number,host_ip,host_port,slab_buf_bytes,slab_partition,bandwidth,file_size_bytes,time_taken_sec,overall_cpu,overall_mem
0,0,10.0.0.50,5202,1073741824,4,34.56 Gb,1.073742e+09,0.248558,63.752396,0.943125
1,1,10.0.0.50,5202,1073741824,4,34.54 Gb,1.073742e+09,0.248713,65.253389,0.925443
2,2,10.0.0.50,5202,1073741824,4,34.95 Gb,1.073742e+09,0.245784,62.988425,0.941397
3,3,10.0.0.50,5202,1073741824,4,33.46 Gb,1.073742e+09,0.256729,64.137331,0.925443
4,4,10.0.0.50,5202,1073741824,4,31.05 Gb,1.073742e+09,0.276656,67.037435,0.91585
...,...,...,...,...,...,...,...,...,...,...
95,95,10.0.0.50,5202,1073741824,4,33.41 Gb,1.073742e+09,0.257124,89.670386,0.744526
96,96,10.0.0.50,5202,1073741824,4,33.84 Gb,1.073742e+09,0.253842,89.9683,0.75829
97,97,10.0.0.50,5202,1073741824,4,34.08 Gb,1.073742e+09,0.252080,89.788217,0.755474
98,98,10.0.0.50,5202,1073741824,4,34.10 Gb,1.073742e+09,0.251901,89.984984,0.748488


In [743]:
sender_df.to_csv('1gb_logs_cleansed.csv')